In [2]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.chrome.service import Service # Import Service class to specify ChromeDriver location
from webdriver_manager.chrome import ChromeDriverManager
from selenium.common.exceptions import TimeoutException
import time
from bs4 import BeautifulSoup
import pandas as pd

In [3]:
# Replace with the path to your ChromeDriver executable
options = webdriver.ChromeOptions()
options.add_argument("--no-sandbox")
options.add_argument("--disable-dev-sh-usage")
# options.add_argument("--headless")
driver = webdriver.Chrome(service=Service(ChromeDriverManager().install()), options=options)

driver.get('https://www.amazon.com/')
# Open Facebook login page

In [4]:
df = pd.read_csv('product_links/women/women_clothing.csv')
links = df['links']
links

0       https://www.amazon.com/sspa/click?ie=UTF8&spc=...
1       https://www.amazon.com/Dress-Population-Tiffan...
2       https://www.amazon.com/Richards-Womens-Duster-...
3       https://www.amazon.com/Maidenform-Seamless-Lon...
4       https://www.amazon.com/sspa/click?ie=UTF8&spc=...
                              ...                        
1995    https://www.amazon.com/Amazon-Essentials-Stand...
1996    https://www.amazon.com/Lucky-Brand-Womens-CAMO...
1997    https://www.amazon.com/Cosabella-Womens-Evolut...
1998    https://www.amazon.com/sspa/click?ie=UTF8&spc=...
1999    https://www.amazon.com/NIGHTWAY-Womens-Sleevel...
Name: links, Length: 2000, dtype: object

In [5]:
reviewers = []
ids = []

for i, link in enumerate(links):
    if i < len(links):
        print(i)
        try:
            driver.get(link)
            time.sleep(1)
            rating_link = driver.find_element(By.ID, "acrCustomerReviewLink")
            rating_link.click()
            see_more_div = driver.find_element(By.ID, "reviews-medley-footer")
            anchor_tag = see_more_div.find_element(By.TAG_NAME, "a")
            driver.get(anchor_tag.get_attribute("href"))
            count = 0
            while True:
                try:
                    time.sleep(1)
                    review_div = driver.find_element(By.ID, "cm_cr-review_list")
                    reviewer_profiles = review_div.find_elements(By.CLASS_NAME, "a-profile")
                    count += len(reviewer_profiles)
                    for profile in reviewer_profiles:
                        url = profile.get_attribute("href")
                        if url != "":
                            id = url.split('/ref')[0].split('account.')[1]
                            reviewers.append(url)
                            ids.append(id)
                    see_more_link = driver.find_element(By.CLASS_NAME, "a-last")
                    a = see_more_link.find_element(By.TAG_NAME, "a")
                    driver.get(a.get_attribute("href"))

                except:
                    break
            print(count)
            print(driver.title)
            time.sleep(1)
        except:
            print("Failed to load reviews")
            continue
    else:
        break
print("complete")

0


Failed to load reviews
1
18
Amazon.com: Customer reviews: Dress the Population Women&#39;s Tiffany Asymmetrical Bow Neckline Bodycon Midi Dress
2
107
Amazon.com: Customer reviews: R&amp;M Richards Womens Plus Sequined Lace Pant Suit
3
104
Amazon.com: Customer reviews: Maidenform Women&#39;s M Seamless Lift Bralette, Wireless Long-line Convertible Tank-style Bralette
4
9
Amazon.com: Customer reviews: Simone Perele womens Muse Contour Plunge Bra, Peau Rose, 30F US
5
9
Amazon.com: Customer reviews: Simone Perele womens Muse Contour Plunge Bra, Peau Rose, 30F US
6
3
Amazon.com: Customer reviews: HUDSON Women&#39;s Jodie Wide Leg
7
103
Amazon.com: Customer reviews: Amazon Essentials Women&#39;s Jersey Regular-Fit Sleeveless v-Neck Midi Dress (Previously Daily Ritual)
8
107
Amazon.com: Customer reviews: Chef Works Women&#39;s Springfield Chef Coat
9
7
Amazon.com: Customer reviews: FERXXO Feid Official Merch Ferxxocalipsis Tour T-Shirt
10
102
Amazon.com: Customer reviews: Maidenform Women&#39

In [6]:
len(reviewers), len(list(set(reviewers))) 

(64354, 51237)

In [8]:
df = pd.DataFrame(columns=['links'])

df['links'] = list(set(reviewers))
# df['reviewer_id'] = ids

df.to_csv('reviewers_links/women_clothing_reviewers.csv', index=False)